In [3805]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

# 1) Explorando o Dataset

O Dataset escolhido foi do site kaggle: https://www.kaggle.com/datasets/khanghunhnguyntrng/football-players-transfer-fee-prediction-dataset



In [3806]:
df = pd.read_csv("final_data.csv")
df_intocado = df

In [3807]:
df.head()

,player,team,name,position,height,age,appearance,goals,assists,yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,/david-de-gea/profil/spieler/59377,Manchester United,David de Gea,Goalkeeper,189.0,32.0,104,0.000000,0.000000,0.009585,...,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,/jack-butland/profil/spieler/128899,Manchester United,Jack Butland,Goalkeeper,196.0,30.0,15,0.000000,0.000000,0.069018,...,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,/tom-heaton/profil/spieler/34130,Manchester United,Tom Heaton,Goalkeeper,188.0,37.0,4,0.000000,0.000000,0.000000,...,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,/lisandro-martinez/profil/spieler/480762,Manchester United,Lisandro Martínez,Defender Centre-Back,175.0,25.0,82,0.028090,0.056180,0.224719,...,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,/raphael-varane/profil/spieler/164770,Manchester United,Raphaël Varane,Defender Centre-Back,191.0,30.0,63,0.017889,0.017889,0.053667,...,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0


In [3808]:
df.columns

Index(['player', 'team', 'name', 'position', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded', 'winger'],
      dtype='object')

In [3809]:
df.shape
# Muitas linhas será interessante para predizer um valor de um jogador fictício

(10754, 22)

Para aprimorar a avaliação do modelo de previsão, as estatísticas, a saber, "gols", "assistências", "cartões amarelos", "segundos cartões amarelos", "cartões vermelhos", "gols sofridos" e "gols sem sofrer gols", foram transformadas para uma base por 90 minutos. Essa transformação envolveu a divisão de cada valor estatístico pelo valor correspondente por 90 minutos (calculado como minutos jogados divididos por 90).

É importante ressaltar que os dados são de 2 temporadas 2021-2022 e 2022-2023, então os valores serão diferentes de hoje em dia (2025).

De acordo com a análise das colunas do dataset, os melhores parâmetros para realizar uma predição do valor de mercado de um jogador são:
1. Time ✔
2. Posição ✔
3. Altura ✔
4. Idade ✔
5. Aparições (em partidas) ✔
6. Gols ✔
7. Assistências ✔
8. Cartão Amarelo ✔
9. 2o Cartão Amarelo
10. Cartão Vermelho
11. Gols Concedidos ✔
12. "Clean Sheets" ✔
13. Minutos jogados ✔
14. Dias Machucado ✔
15. Partidas Machucado ✔
16. Troféus ✔
17. Valor atual (euro)

Algumas colunas que não estão na lista serão apenas para identificação e outras métricas.

As informações de Time poderão ser agrupadas em ligas, então a ideia será criar uma nova coluna representando a liga em que o jogador jogar, pois possivelmente por time ficaria muito complicado (?)



## 2) Embasamento teórico do modelo preditivo:

Iremos utilizar os dados já obtidos para prever o valor de um jogador fictício, que esteja presente em algum desses clubes/ligas, dando a ele todas as características listadas anteriormente como parâmetros de entrada, e de saída iremos retornar o seu valor de mercado estimado. Algo interessante que podemos fazer também é remover um jogador arbitrariamente do dataset e testar o modelo nele, removendo seu valor de mercado, para avaliar se o modelo está se saindo bem em sua predição.

A princípio, iremos utilizar o seguinte raciocínio: Teremos os valores dos jogadores como pontos em um gráfico de N dimensões. Cada eixo irá corresponder a um parâmetro escolhido. A ideia será realizar a média das distâncias dos pontos mais próximos do ponto que queremos encontrar (no caso o valor do jogador fictício). A média da distância será o valor do jogador em questão. O nome desse modelo é o KNN simples, e irei explicar passo a passo de como ele funciona:

O modelo KNN simples é uma técnica que leva em consideração a proximidade entre os dados para a realização de predições. Dados similares tendem a estar concentrados na mesma região no espaço de dispersão dos dados. Uma boa forma de explicar esse modelo é pensar em um espaço 2D, onde temos o eixo x e y. Nesse espaço, já possuímos pontos marcados nele (que vem dos dados do dataset, mais especificamente do conjunto de validação) e vamos inserir um novo ponto, onde não temos seu valor. A partir dele, iremos calcular a distância desse nosso novo ponto com os outros pontos vizinhos dele. Essa distância é a distância euclidiana:

suponha $x_0 , y_0$ sendo as coordenadas do ponto que colocamos

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)²}$

Para um espaço de dimensão 3 teríamos:

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)² + (z_0 - z_1)²}$

Para um espaço de dimensão N teríamos:

- d = $\sqrt{(x_0 - x_1)² + (y_0 - y_1)² + (z_0 - z_1)² + ... + (n_0 - n_1)²}$

Lembrando que essas fórmulas são para visualização da distância entre o ponto que escolhemos, com um outro ponto já colocado no nosso espaço. No nosso modelo, iremos usar 3 vizinhos, logo, teríamos que realizar essa fórmula para esses 3 pontos. Algo como:

- d_total = $\sum_{i=1}^{j}\sqrt{(x_0 - x_i)² + (y_0 - y_i)² + (z_0 - z_i)² + ... + (n_0 - n_i)²}$

Sendo $j = 3$ pois estamos pensando em 3 vizinhos.

Com as distâncias calculadas, realizamos a média e chegamos em um resultado. Esse resultado é o valor que estamos prevendo, sendo então o valor de mercado estimado do nosso jogador.

Temos que ter cuidado com as escalas dos eixos, esse será um grande desafio nessa modelagem. Escolher a melhor forma de representar os dados tem um grande impacto no resultado final.

### 2.1) Separando times em ligas

A ideia será transformar a coluna de times em uma coluna de ligar, numerada de 1 a 20 (pois temos 20 ligas no dataset). Tal fato irá contribuir para a regressão, pois jogadores de mesma liga costumam ter preços mais similares. Um jogador da Premier League tem uma chance muito mais alta de valer mais que um jogador da liga australiana, por exemplo. 

In [3810]:
# Cada liga tem 20 times em um geral, depende da liga
liga_inglesa = df["team"].unique()[0:20]
liga_alema = df["team"].unique()[20:38]
liga_espanhola = df["team"].unique()[38:58]
liga_italiana = df["team"].unique()[58:78]
liga_francesa = df["team"].unique()[78:98]
liga_holandesa = df["team"].unique()[98:116]
liga_brasileira = df["team"].unique()[116:136]
liga_portuguesa = df["team"].unique()[136:154]
liga_mexicana = df["team"].unique()[154:172]
liga_russa = df["team"].unique()[172:188]
liga_2ainglesa = df["team"].unique()[188:212]
liga_turca = df["team"].unique()[212:231]
liga_austriaca = df["team"].unique()[231:243]
liga_estadounidense = df["team"].unique()[243:272]
liga_argentina = df["team"].unique()[272:300]
liga_japonesa = df["team"].unique()[300:318]
liga_arabe = df["team"].unique()[318:334]
liga_coreana = df["team"].unique()[334:346]
liga_sulafricana = df["team"].unique()[346:361]
liga_australiana = df["team"].unique()[361:374]

In [3811]:
# Criando um mapeamento de times para ligas
liga_map = {
    **{team: 1 for team in liga_inglesa},
    **{team: 2 for team in liga_alema},
    **{team: 3 for team in liga_espanhola},
    **{team: 4 for team in liga_italiana},
    **{team: 5 for team in liga_francesa},
    **{team: 6 for team in liga_holandesa},
    **{team: 7 for team in liga_brasileira},
    **{team: 8 for team in liga_portuguesa},
    **{team: 9 for team in liga_mexicana},
    **{team: 10 for team in liga_russa},
    **{team: 11 for team in liga_2ainglesa},
    **{team: 12 for team in liga_turca},
    **{team: 13 for team in liga_austriaca},
    **{team: 14 for team in liga_estadounidense},
    **{team: 15 for team in liga_argentina},
    **{team: 16 for team in liga_japonesa},
    **{team: 17 for team in liga_arabe},
    **{team: 18 for team in liga_coreana},
    **{team: 19 for team in liga_sulafricana},
    **{team: 20 for team in liga_australiana},
}

# Adicionando uma nova coluna 'liga' ao dataframe
df['league'] = df['team'].map(liga_map)

# Verificando se temos times sem mapeamento
times_sem_liga = df[df['league'].isnull()]['team'].unique()
if len(times_sem_liga) > 0:
    print("Times sem liga mapeada:", times_sem_liga)

# Preenchendo times sem liga com um valor padrão (opcional)
df['league'] = df['league'].fillna(0)

# Não tivemos nenhum que não estivesse mapeado, pois o dataset está bem tratado

In [3812]:
df["position"].unique()

array(['Goalkeeper', 'Defender Centre-Back', 'Defender Left-Back',
       'Defender Right-Back', 'midfield-DefensiveMidfield',
       'midfield-CentralMidfield', 'midfield-AttackingMidfield',
       'Attack-LeftWinger', 'Attack-RightWinger', 'Attack Centre-Forward',
       'midfield-RightMidfield', 'midfield-LeftMidfield',
       'Attack-SecondStriker', 'midfield', 'Attack', 'Defender'],
      dtype=object)

Também fizemos um encoded da posição, de forma mais específica

In [3813]:
le = LabelEncoder()
df["position_encoded_specific"] = le.fit_transform(df["position"])
dict(zip(le.classes_, le.transform(le.classes_)))

{'Attack': 0,
 'Attack Centre-Forward': 1,
 'Attack-LeftWinger': 2,
 'Attack-RightWinger': 3,
 'Attack-SecondStriker': 4,
 'Defender': 5,
 'Defender Centre-Back': 6,
 'Defender Left-Back': 7,
 'Defender Right-Back': 8,
 'Goalkeeper': 9,
 'midfield': 10,
 'midfield-AttackingMidfield': 11,
 'midfield-CentralMidfield': 12,
 'midfield-DefensiveMidfield': 13,
 'midfield-LeftMidfield': 14,
 'midfield-RightMidfield': 15}

### 2.2) Realizando a separação das colunas que serão levadas em consideração para a regressão

Removendo colunas que não serão utilizadas

In [3814]:
# df = df.drop(["winger","player","highest_value", "name"],axis=1)

# Selecionar apenas as colunas numéricas relevantes
columns = [
    'league', 'position_encoded_specific', 'height', 'age', 'appearance',
    'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
    'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
    'games_injured', 'award', 'current_value', 'highest_value',
    'position_encoded', 'winger'
]
df = df[columns]
df

,league,position_encoded_specific,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,9,189.000000,32.0,104,0.000000,0.000000,0.009585,0.0,0.000000,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,9,196.000000,30.0,15,0.000000,0.000000,0.069018,0.0,0.000000,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,9,188.000000,37.0,4,0.000000,0.000000,0.000000,0.0,0.000000,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,6,175.000000,25.0,82,0.028090,0.056180,0.224719,0.0,0.000000,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,1,6,191.000000,30.0,63,0.017889,0.017889,0.053667,0.0,0.000000,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10749,20,3,181.240353,20.0,16,0.175953,0.087977,0.263930,0.0,0.000000,0.000000,0.000000,1023,0,0,0,75000,75000,4,1
10750,20,1,190.000000,24.0,26,0.372671,0.186335,0.186335,0.0,0.000000,0.000000,0.000000,1449,102,18,0,300000,300000,4,0
10751,20,1,181.240353,19.0,20,0.375000,0.000000,0.187500,0.0,0.000000,0.000000,0.000000,960,0,0,0,50000,50000,4,0
10752,20,1,181.240353,20.0,17,0.312139,0.104046,0.000000,0.0,0.104046,0.000000,0.000000,865,0,0,0,50000,50000,4,0


In [3815]:
df.head(4)

,league,position_encoded_specific,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,9,189.0,32.0,104,0.00000,0.00000,0.009585,0.0,0.0,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,9,196.0,30.0,15,0.00000,0.00000,0.069018,0.0,0.0,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,9,188.0,37.0,4,0.00000,0.00000,0.000000,0.0,0.0,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,6,175.0,25.0,82,0.02809,0.05618,0.224719,0.0,0.0,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0


# 3) Separação de conjunto de TREINO TESTE E VALIDAÇÃO

- Treinamento: onde o modelo aprende.
- Validação: onde ajustamos hiperparâmetros.
- Teste: onde avaliamos o desempenho final. Esse conjunto deve ficar intocado até o final.

No nosso modelo, treinaremos utilizando:

- X_train, y_train.

Testaremos o modelo usando

- X_val, y_val, variando k.

Quando tiver o melhor modelo (melhor k), aí sim:

- Usa X_test, y_test uma única vez.
- Isso vai te dizer se o modelo está bom mesmo com dados nunca vistos.

#### 3.1) Separar X e y

X será o conjunto de dados que incluirão todas as colunas, exceto a que queremos prever

y será o conjunto de dados que apenas inclue a coluna "current_value"

In [3816]:
X = df[columns].drop(columns=["current_value"])
y = df["current_value"]

#### 3.2) Separar treino+validação e teste

Seguindo o que foi dito na seção 1, temos X e y, e agora iremos separar em teste, primeiramente, para ambos os conjuntos

In [3817]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3818]:
X_temp.shape

(8603, 19)

In [3819]:
X_test.shape

(2151, 19)

In [3820]:
y_temp.shape

(8603,)

In [3821]:
y_test.shape

(2151,)

#### 3.3) Separar treino e validação

Usamos o X_temp e y_temp para fazermos a ultima separação necessária (já que o train_test_split apéns separa em 2 partes). Essa separação irá resultar em um conjunto de dados dividido na seguinte porcentagem:

- 60% treino, 20% validação, 20% teste

In [3822]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [3823]:
X_train.shape

(6452, 19)

In [3824]:
X_val.shape

(2151, 19)

In [3825]:
y_train.shape

(6452,)

In [3826]:
y_val.shape

(2151,)

#### 3.4) Normalização feita apenas com o treino

apenas o treino realiza a normalização, pois não queremos que esse conjunto use informações futuras.

In [3827]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Após termos os dados normalizados, podemos aplicar o KNN simples nesse dataset e obter o valor de mercado do jogador em questão

In [3828]:
# Previsão no conjunto de validação
k = 5
preds_val = []

# Percorrendo cada jogador no conjunto de validação
for i in range(len(X_val_scaled)):
    # Distância euclidiana dos pontos vizinhos
    distances = euclidean_distances(X_train_scaled, [X_val_scaled[i]])

    # Aqui selecionamos os k vizinhos mais próximos
    indices = np.argsort(distances.ravel())[:k]

    # Pegamos os valores reais de mercados dos k vizinhos e fazemos a média, aqui é a previsão
    mean_value = y_train.iloc[indices].mean()
    preds_val.append(mean_value)

# Avaliando o erro no conjunto de validação
val_error = mean_absolute_error(y_val, preds_val)
print(f"Erro médio absoluto na validação: €{val_error:.2f}")


Erro médio absoluto na validação: €1794113.44


In [3829]:
from sklearn.metrics import mean_absolute_error
import numpy as np

mae = mean_absolute_error(y_val, preds_val)
mape = np.mean(np.abs((y_val - preds_val) / (y_val + 1e-8))) * 100

print(f"Erro absoluto médio (MAE): €{mae:,.2f}")
print(f"Erro percentual médio (MAPE): {mape:.2f}%")


Erro absoluto médio (MAE): €1,794,113.44
Erro percentual médio (MAPE): 53054393305638.24%


In [3830]:
numerador = np.abs(y_val - preds_val)
denominador = (np.abs(y_val) + np.abs(preds_val)) / 2 + 1e-8
smape = np.mean(numerador / denominador) * 100
print(f"SMAPE: {smape:.2f}%")


SMAPE: 72.94%


In [3831]:
df

,league,position_encoded_specific,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,1,9,189.000000,32.0,104,0.000000,0.000000,0.009585,0.0,0.000000,1.217252,0.335463,9390,42,5,13,15000000,70000000,1,0
1,1,9,196.000000,30.0,15,0.000000,0.000000,0.069018,0.0,0.000000,1.242331,0.207055,1304,510,58,1,1500000,22000000,1,0
2,1,9,188.000000,37.0,4,0.000000,0.000000,0.000000,0.0,0.000000,0.616438,0.924658,292,697,84,4,600000,6000000,1,0
3,1,6,175.000000,25.0,82,0.028090,0.056180,0.224719,0.0,0.000000,0.000000,0.000000,6408,175,22,9,50000000,50000000,2,0
4,1,6,191.000000,30.0,63,0.017889,0.017889,0.053667,0.0,0.000000,0.000000,0.000000,5031,238,51,21,40000000,80000000,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10749,20,3,181.240353,20.0,16,0.175953,0.087977,0.263930,0.0,0.000000,0.000000,0.000000,1023,0,0,0,75000,75000,4,1
10750,20,1,190.000000,24.0,26,0.372671,0.186335,0.186335,0.0,0.000000,0.000000,0.000000,1449,102,18,0,300000,300000,4,0
10751,20,1,181.240353,19.0,20,0.375000,0.000000,0.187500,0.0,0.000000,0.000000,0.000000,960,0,0,0,50000,50000,4,0
10752,20,1,181.240353,20.0,17,0.312139,0.104046,0.000000,0.0,0.104046,0.000000,0.000000,865,0,0,0,50000,50000,4,0


Aqui iremos aplicar o modelo já treinado (com os dados de treino) para prever o valor de um jogador novo, no que resulta em uma simulação em uma situação real de uso.

In [3832]:
X_train.columns

Index(['league', 'position_encoded_specific', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'highest_value', 'position_encoded',
       'winger'],
      dtype='object')

In [3833]:
# Jogador fictício (ex: Lisandro Martínez)
jogador_ficticio = {
    "league": 1, "position_encoded_specific": 6, "height": 175, "age": 25, "appearance": 82, "goals": 0.02809, 
    "assists": 0.05618, "yellow cards": 0.224719, "second yellow cards": 0.0, "red cards": 0.0,
    "goals conceded": 0, "clean sheets": 0, "minutes played": 6408, "days_injured": 175,
    "games_injured": 22, "award": 9, "highest_value": 50000000,
    "position_encoded": 2, "winger": 0, 
}

# Transformando o dicionário em dataframe e normalizando os dados
# Para evitar erros de escala
jogador_ficticio_df = pd.DataFrame([jogador_ficticio])
normalized_jogador = scaler.transform(jogador_ficticio_df)

# Distância euclidiana dos pontos vizinhos
distances = euclidean_distances(X_train_scaled, normalized_jogador)

# Aqui selecionamos os k vizinhos mais próximos
indices = np.argsort(distances.ravel())[:k]

# Fazendo a média das distâncias usando o conjunto de treino
predicted_value = y_train.iloc[indices].mean()

print(f"Valor predito para o jogador Lisandro Martínez: €{predicted_value:,.2f}")
print(f"Valor real: €{df.iloc[3]['current_value']:,.2f}")
erro = abs(predicted_value - df.iloc[3]["current_value"])
print(f"Erro: €{erro:,.2f}")


Valor predito para o jogador Lisandro Martínez: €49,400,000.00
Valor real: €50,000,000.00
Erro: €600,000.00


OBS: eu testei antes sem a liga, e o valor do Lisandro Martinez dava 5milhoes de diferença do valor real, depois de adicionar a liga foi para 2milhoões de erro, o que é uma boa melhora. Depois de adicionar todas as colunas numéricas ficamos com 1milhão de diferença, melhor ainda! Isso tudo eu tinha feito sem os conjuntos de treino, teste e validação. Após essa adição tivemos uma diferença de 600.000, o que melhorou ainda mais

### 4) Criando um jogador fictício

Primeiramente, para criarmos um jogador temos que atender às métricas do dataset. Temos que lembrar que algumas colunas são em uma métrica que chamam de "por 90 minutos". Para termos uma comparação, por serem 2 temporadas, temos os gols de lionel Messi nas temporadas 21/22 e 22/23 (seu tempo no inter miami não está contado aqui)

(GOLS / MINUTOS JOGADOS) * 90

por exemplo, Messi fez 32 gols, pois 

$(gols / 6509) * 90 = 0.442464$

$90 * gols / 6509 = 0.442464$

$90 * gols = 2879,998176$

$gols = 31,999$

$gols = 32$

É importante ressaltar que os dados são de 2 temporadas 2021-2022 e 2022-2023, então os valores serão diferentes de hoje em dia (2025). Nas próximas celulas teremos os dados de 1 jogador para termos como base, o valor máximo de minutos jogados e uma tabela com as ligas para podermos esolher 

Seguem os dados do Messi 🐐

In [3834]:
df_a = df_intocado[df_intocado['name'].str.match('Lionel Messi', na = False)]
df_a

,player,team,name,position,height,age,appearance,goals,assists,yellow cards,...,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger,league,position_encoded_specific
2643,/lionel-messi/profil/spieler/28003,Paris Saint-Germain,Lionel Messi,Attack-RightWinger,170.0,35.0,75,0.442464,0.483945,0.013827,...,6509,167,30,92,45000000,180000000,4,1,5,3


In [3835]:
# Valor máximo de minutos jogados
df_intocado['minutes played'].max()

9510

Tabela de Ligas:

| Código | Liga                                |
|--------|-------------------------------------|
| 1      | 🏴󠁧󠁢󠁥󠁮󠁧󠁿 Liga Inglesa (Premier League)          |
| 2      | 🇩🇪 Liga Alemã (Bundesliga)                |
| 3      | 🇪🇸 Liga Espanhola (La Liga)               |
| 4      | 🇮🇹 Liga Italiana (Serie A)                |
| 5      | 🇫🇷 Liga Francesa (Ligue 1)                |
| 6      | 🇳🇱 Liga Holandesa (Eredivisie)           |
| 7      | 🇧🇷 Liga Brasileira (Brasileirão)          |
| 8      | 🇵🇹 Liga Portuguesa (Liga Portugal)        |
| 9      | 🇲🇽 Liga Mexicana (Liga MX)               |
| 10     | 🇷🇺 Liga Russa (Russian Premier League)    |
| 11     | 🏴󠁧󠁢󠁥󠁮󠁧󠁿 2ª Liga Inglesa (Championship)         |
| 12     | 🇹🇷 Liga Turca (Süper Lig)                 |
| 13     | 🇦🇹 Liga Austríaca (Bundesliga Austria)    |
| 14     | 🇺🇸 Liga Estadunidense (MLS)               |
| 15     | 🇦🇷 Liga Argentina (Primera División)      |
| 16     | 🇯🇵 Liga Japonesa (J1 League)              |
| 17     | 🇸🇦 Liga Árabe (Saudi Pro League)          |
| 18     | 🇰🇷 Liga Coreana (K League)                |
| 19     | 🇿🇦 Liga Sul-Africana (PSL)                |
| 20     | 🇦🇺 Liga Australiana (A-League)            |

| Código | Posição                          |
|--------|----------------------------------|
| 0      | Attack                           |
| 1      | Attack Centre-Forward            |
| 2      | Attack-LeftWinger                |
| 3      | Attack-RightWinger               |
| 4      | Attack-SecondStriker             |
| 5      | Defender                         |
| 6      | Defender Centre-Back             |
| 7      | Defender Left-Back               |
| 8      | Defender Right-Back              |
| 9      | Goalkeeper                       |
| 10     | Midfield                         |
| 11     | Midfield-AttackingMidfield       |
| 12     | Midfield-CentralMidfield         |
| 13     | Midfield-DefensiveMidfield       |
| 14     | Midfield-LeftMidfield            |
| 15     | Midfield-RightMidfield           |

#### 4.1) Criando nosso jogador

Podemos preencher a célula seguinte com o nome e alguns dados

In [3836]:
# Função para converter os valores mais "humanos" na métrica usada pelo dataset
def calcula_por90(dado):
    res = (dado/7000) * 90
    return res

In [3837]:
# Jogador fictício
nome_jogador = "Rodrigo Guzmán"

jogador_ficticio = {
    "league": 1, "position_encoded_specific": 1, "height": 179, "age": 22, "appearance": 100, "goals": calcula_por90(30), 
    "assists": calcula_por90(40), "yellow cards": calcula_por90(10), 
    "second yellow cards": calcula_por90(1), "red cards": calcula_por90(1),
    "goals conceded": calcula_por90(0), "clean sheets": calcula_por90(0), 
    "minutes played": 7000, "days_injured": 200, "games_injured": 16, "award": 10, 
    "highest_value": 80000000, "position_encoded": 2, "winger": 0
}

# Transformando o dicionário em dataframe e normalizando os dados
# Para evitar erros de escala
jogador_ficticio_df = pd.DataFrame([jogador_ficticio])
normalized_jogador = scaler.transform(jogador_ficticio_df)

# Distância euclidiana dos pontos vizinhos
distances = euclidean_distances(X_train_scaled, normalized_jogador)

# Aqui selecionamos os k vizinhos mais próximos
indices = np.argsort(distances.ravel())[:k]

#Fazendo a média das distâncias usando o conjunto de treino
predicted_value = y_train.iloc[indices].mean()

# Calcular diferença absoluta entre os valores reais e o valor previsto
df_intocado["diff_predicted"] = abs(df_intocado["current_value"] - predicted_value)

# Encontrando o jogador com valor mais próximo
jogador_proximo = df_intocado.loc[df_intocado["diff_predicted"].idxmin()]

print(f"Valor predito para o jogador {nome_jogador}: €{predicted_value:,.2f}")
if jogador_proximo['name'] == 'Antony':
    print(f"Você vale tanto quanto o 🐐 {jogador_proximo['name']} (€{jogador_proximo['current_value']:,.2f})")
else:
    print(f"Você vale tanto quanto o {jogador_proximo['name']} (€{jogador_proximo['current_value']:,.2f})")

df_intocado = df_intocado.drop(columns=['diff_predicted'])


Valor predito para o jogador Rodrigo Guzmán: €69,000,000.00
Você vale tanto quanto o 🐐 Antony (€70,000,000.00)


In [3838]:
print(df.iloc[indices][["goals", "assists", "current_value"]])

         goals   assists  current_value
3778  0.000000  0.176125        1200000
4617  0.063627  0.000000         800000
5257  0.281739  0.250435        1500000
1293  0.432692  0.086538        1000000
1865  0.026362  0.000000        6000000


Dados do jogador que vale tanto quanto você:

In [3839]:
df_a = df_intocado[df_intocado['name'].str.match(f"{jogador_proximo['name']}", na = False)]
df_a

,player,team,name,position,height,age,appearance,goals,assists,yellow cards,...,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger,league,position_encoded_specific
27,/antony/profil/spieler/602105,Manchester United,Antony,Attack-RightWinger,172.0,23.0,80,0.335252,0.228581,0.182865,...,5906,196,22,5,70000000,75000000,4,1,1,3
4110,/antony/profil/spieler/798687,FC Arouca,Antony,Attack-LeftWinger,185.0,21.0,63,0.192565,0.072212,0.144424,...,3739,0,0,0,400000,400000,4,1,8,2
4459,/antony-silva/profil/spieler/153235,Puebla FC,Antony Silva,Goalkeeper,190.0,39.0,71,0.000000,0.000000,0.154930,...,6390,0,0,0,500000,900000,1,0,9,9


# 5) Conclusões

O modelo possui alguns problemas, e alguns dos motivos que pesquisei na internet diziam o seguinte: Um erro alto é comum em um contexto de predição de valor de mercado de jogadores, que é um problema:

- Altamente volátil — dois jogadores com atributos parecidos podem ter valores muito diferentes dependendo de hype, lesões, clube, agente, etc.

- Com distribuição desigual — há muitos jogadores baratos e poucos muito caros (isso distorce bastante os erros percentuais).

- Baseado em KNN — que é um método simples e que não generaliza tão bem quanto modelos mais complexos."

Mas sinto que as porcentagens encontradas nos erros são elevadas pois os valores dos jogadores são muito altos, então isso pode ser um fator que influencia. 

Por mais problemático que esteja nessas métricas, o modelo consegue identificar que o mesmo jogador agindo em diferentes ligas modifica seu valor de mercado, por exemplo. A idade também é algo que afeta o valor do jogador, então ele condiz com a realidade em alguns aspectos. Esperar que um modelo relativamente simples desse tenha uma baixa taxa de erro é irreal, pensando nos fatores mencionados anteriormente. Possivelmente tratamentos de outliers melhorem essas métricas

# 6) Referências

1. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
2. https://www.youtube.com/watch?v=gJK4fmCvcWY
3. https://www.youtube.com/watch?v=HJAChG-GRyA
4. https://www.youtube.com/watch?v=zvmbB3315Ko&t=436s
5. https://pandas.pydata.org/docs/
6. https://scikit-learn.org/stable/